### Chapter 2 of Spark: The Definitive Guide

In [ ]:
# Variable available at start time
spark

Out[3]:

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.0.1 
 Master 
 local[8] 
 AppName 
 Databricks Shell

In [ ]:
# Create range of numbers
myRange = spark.range(1000).toDF('number')

In [ ]:
divisBy2 = myRange.where("number % 2 = 0")

In [ ]:
divisBy2.count()

Out[7]: 500

In [ ]:
# Dislay path to data loaded via GUI as a table
display(dbutils.fs.ls("/FileStore/tables/"))

path,name,size
dbfs:/FileStore/tables/2015_summary.csv,2015_summary.csv,7080


In [ ]:
# Load flight data
flightData2015 = spark\
  .read\
  .option("inferSchema", "true")\
  .option("header","true")\
  .csv("dbfs:/FileStore/tables/2015_summary.csv")

In [ ]:
# Print first 3 rows
flightData2015.take(3)

Out[19]: [Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Romania', count=15),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Croatia', count=1),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Ireland', count=344)]

In [ ]:
# Inspect spark's plan
flightData2015.sort("count").explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Sort [count#926 ASC NULLS FIRST], true, 0
 +- Exchange rangepartitioning(count#926 ASC NULLS FIRST, 200), true, [id=#628]
 +- FileScan csv [DEST_COUNTRY_NAME#924,ORIGIN_COUNTRY_NAME#925,count#926] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex[dbfs:/FileStore/tables/2015_summary.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string,ORIGIN_COUNTRY_NAME:string,count:int>

In [ ]:
# Test modifying the number of partitions
spark.conf.set("spark.sql.shuffle.partition","4")
flightData2015.sort("count").take(2)

Out[32]: [Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Singapore', count=1),
 Row(DEST_COUNTRY_NAME='Moldova', ORIGIN_COUNTRY_NAME='United States', count=1)]

In [ ]:
flightData2015.createOrReplaceTempView("flight_Data_2015")

In [ ]:
# Query DataFrame via SQL
sqlWay = spark.sql("""
SELECT DEST_COUNTRY_NAME, count(1)
FROM flight_Data_2015
GROUP BY DEST_COUNTRY_NAME
""")

sqlWay.explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[DEST_COUNTRY_NAME#924], functions=[finalmerge_count(merge count#977L) AS count(1)#972L])
 +- Exchange hashpartitioning(DEST_COUNTRY_NAME#924, 200), true, [id=#761]
 +- HashAggregate(keys=[DEST_COUNTRY_NAME#924], functions=[partial_count(1) AS count#977L])
 +- FileScan csv [DEST_COUNTRY_NAME#924] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex[dbfs:/FileStore/tables/2015_summary.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string>

In [ ]:
# Query DataFrame directly
dataFrameWay = flightData2015\
  .groupBy("DEST_COUNTRY_NAME")\
  .count()

dataFrameWay.explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[DEST_COUNTRY_NAME#924], functions=[finalmerge_count(merge count#989L) AS count(1)#984L])
 +- Exchange hashpartitioning(DEST_COUNTRY_NAME#924, 200), true, [id=#807]
 +- HashAggregate(keys=[DEST_COUNTRY_NAME#924], functions=[partial_count(1) AS count#989L])
 +- FileScan csv [DEST_COUNTRY_NAME#924] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex[dbfs:/FileStore/tables/2015_summary.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string>

In [ ]:
# Get max count value
from pyspark.sql.functions import max
flightData2015.select(max("count")).take(1)

Out[37]: [Row(max(count)=370002)]

In [ ]:
# Aggregate top 5 destination totals
from pyspark.sql.functions import desc
flightData2015\
  .groupBy("DEST_COUNTRY_NAME")\
  .sum("count")\
  .withColumnRenamed("sum(count)","destination_total")\
  .sort(desc("destination_total"))\
  .limit(5)\
  .show()

+-----------------+-----------------+
DEST_COUNTRY_NAME|destination_total|
+-----------------+-----------------+
 United States| 411352|
 Canada| 8399|
 Mexico| 7140|
 United Kingdom| 2025|
 Japan| 1548|
+-----------------+-----------------+